# Limpieza de datos

Para los datos del Estado de México que son historicos se tiene que realizar lo siguiente:


*   Eliminar columnas: Precipitación, Evaporación
*   Si la temperatura máxima y mínima en algún registro es nulo, eliminarlo
*.  Si es nulo alguno de los dos valores de temperatura, calcular el promedio con las dos temperaturas anteriores y las dos temperaturas posteriores



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url = "https://practicas6336904464.blob.core.windows.net/diplomadodc/data/Edo%20Amecameca.xlsx"

In [ ]:
def limpieza(url, export):
  df = pd.read_excel(url)
  # df.head()
  # Transformación de columnas a numerico
  columnas = ['Precipitacion', 'Evaporacion', 'TemperaturaMaxima', 'TemperaturaMinima']
  df[columnas] = df[columnas].apply(pd.to_numeric, errors="coerce")

  # Tranformación de columna a fecha
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors="coerce")

  # Eliminar datos cuyas temperaturas son nulos
  df = df.dropna(subset=['TemperaturaMaxima','TemperaturaMinima'], how = "all")
  export = export + '.csv'
  df.to_csv(export, index = False)
  try:
    with open(export,'r') as file:
      return True
  except FileNotFoundError:
    return False

In [ ]:
limpieza(url,'Amecameca')

<ipython-input-21-07fb025eb3d6>:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Fecha'] = pd.to_datetime(df['Fecha'], errors="coerce")


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import chardet

import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import os

In [ ]:
# Autenticación y creación del cliente PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# ID del archivo XLSX
cerro = '1oasPlUgkceq517yNrmTnLNYf88_ksI9F'
amecameca = '1IfirSXW7x8fLf4Fzogw2fh921OduyUpb'
laguna = '1tNB3kcimpF8vQ0LYDws39H5omZERSBlB'


In [ ]:
def leer_archivo_desde_drive(file_id):
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile('temp.xlsx')
  df = pd.read_excel('temp.xlsx')
  return df

cerro_df = leer_archivo_desde_drive(cerro)
amecameca_df = leer_archivo_desde_drive(amecameca)
laguna_df = leer_archivo_desde_drive(laguna)

In [ ]:
#Descarga los archivos y los convierte en dataframes

# ID de la carpeta compartida que deseas listar
carpeta_id = '1pvA46f64RB_3CpOXZQ68iMGgWBZkkqs-'

# Lista para almacenar los nombres de los archivos
files_dataframes = []

# Obtener la lista de archivos en la carpeta compartida
file_list = drive.ListFile({'q': "'{}' in parents".format(carpeta_id)}).GetList()

for archivo in file_list:
    try:
        # Descargar el archivo y guardarlo en el directorio actual
        archivo.GetContentFile(archivo['title'])
        # Leer el archivo descargado y crear un DataFrame
        df = pd.read_excel(archivo['title'])
        # Agregar el DataFrame al array
        files_dataframes.append(df)
        print("Archivo descargado y DataFrame creado:", archivo['title'])
    except Exception as e:
        print("Error al descargar y crear DataFrame para el archivo:", archivo['title'])
        print(e)

Archivo descargado y DataFrame creado: Edo Amecameca.xlsx
Archivo descargado y DataFrame creado: PueblaSan Nicolas.xlsx
Archivo descargado y DataFrame creado: laguna.xlsx
Archivo descargado y DataFrame creado: cerro.xlsx
Archivo descargado y DataFrame creado: nevado.xlsx


In [ ]:
def limpiar_datos(df):
    #df.drop(columns=['Precipitacion', 'Evaporacion', 'TemperaturaMaxima', 'TemperaturaMinima'], inplace=True)
    columnas = ['Precipitacion', 'Evaporacion', 'TemperaturaMaxima', 'TemperaturaMinima']
    df[columnas] = df[columnas].apply(pd.to_numeric, errors="coerce")
    #Cambiar el formato de la columna fecha
    #df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors="coerce")
    # Eliminar datos de temperaturas nulas
    #df.dropna(subset=['TemperaturaMaxima', 'TemperaturaMinima'], inplace=True)
    #df['TemperaturaMaxima'] = df['TemperaturaMaxima'].replace('Nulo', 0)
    #df['TemperaturaMinima'] = df['TemperaturaMinima'].replace('Nulo', 0)
    df = df.dropna(subset=['TemperaturaMaxima','TemperaturaMinima','Evaporacion'], how = "all")

    return df

In [ ]:
# Crea un ThreadPoolExecutor con 4 hilos
with ThreadPoolExecutor(max_workers=4) as executor:
    # Ejecuta la función de limpieza para cada archivo
    resultados = list(executor.map(limpiar_datos, files_dataframes))

df_final = pd.concat(resultados, ignore_index = True)

In [ ]:
df_final.to_csv('resultados_modulos.csv', index=False)
# Guarda el DataFrame en un archivo CSV dentro de la carpeta compartida
#final_df.to_csv(os.path.join(shared_folder_path, 'resultados_modulos.csv'), index=False)

print(df_final.head(50))

        Fecha  Precipitacion  Evaporacion  TemperaturaMaxima  \
0  1969-03-15            0.0          NaN               22.5   
1  1969-03-16            0.0          NaN               22.0   
2  1969-03-17            4.3          NaN               13.5   
3  1969-03-18            1.5          NaN               18.5   
4  1969-03-19            0.0          NaN               19.5   
5  1969-03-20            0.0          NaN               22.0   
6  1969-03-21            0.0          NaN               23.0   
7  1969-03-22            0.0          NaN               23.0   
8  1969-03-23            0.0          NaN               23.0   
9  1969-03-24            0.0          NaN               18.0   
10 1969-03-25            0.0          NaN               19.0   
11 1969-03-26            0.0          NaN               18.5   
12 1969-03-27            2.5          NaN               19.0   
13 1969-03-28            0.0          NaN               20.0   
14 1969-03-29            0.0          Na

In [ ]:
print(df_final.isna().sum())

Fecha                    0
Precipitacion          225
Evaporacion          23794
TemperaturaMaxima      721
TemperaturaMinima      312
dtype: int64
